## 1. Package Preparation

In [1]:
#import packages

# clear warnings
import warnings
warnings.simplefilter("ignore")

# import keras data
from keras import backend as K
K.set_image_dim_ordering('tf') # note that we need to have tensorflow dimension ordering still because of the weigths.
print('The backend is:',K.backend())
import tensorflow as tf
print(K.image_dim_ordering()) # should say tf
print(tf.__version__) # tested for 1.11.0
import cv2
import keras
print(keras.__version__) # tested for 2.2.4

Using TensorFlow backend.


The backend is: tensorflow
tf
1.11.0
2.2.4


In [2]:
# Import relevant packages
from __future__ import absolute_import, division, print_function # make it compatible w Python 2
import os
import h5py # to handle weights
import os, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

# to read image
from PIL import Image

# relative keras packages
from keras.models import Sequential
from keras.layers import LSTM, Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation, ZeroPadding2D
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import model_from_json
from keras.preprocessing import image

# useful packages from sklearn
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

from IPython.display import Image, display

# fix random seed for reproducibility
np.random.seed(150)

In [3]:
import cv2

## 2. Read data from preprocessed pickle file

In [4]:
# We have preprocessed our data and saved in a pickle file
# Now we only need to read this pickle file
map_data = pd.read_pickle('position_df.pkl')

In [5]:
## RESIZE

map_data['image'] = map_data['image'].apply(lambda x: cv2.resize(x,(299,299)))

In [6]:
map_data.iloc[0,1].shape

()

The column 'index' in the map_data above is used to help match the location data for the 'location' column.

We only need the data in the 'label' column and 'image' column to train our model, so we drop those two unnecessary columns.

In [7]:
map_data.drop(['location','index'],axis=1,inplace=True)

In [8]:
map_data.shape

(3493, 2)

In [9]:
# Show the head of our map_data
map_data.head()

,label,image
0,3,"[[[68, 43, 39], [75, 51, 47], [84, 60, 57], [9..."
1,4,"[[[67, 38, 37], [69, 37, 36], [70, 38, 38], [7..."
2,4,"[[[65, 51, 59], [70, 55, 78], [86, 75, 84], [9..."
3,4,"[[[106, 92, 100], [99, 87, 95], [89, 74, 84], ..."
4,4,"[[[150, 141, 146], [146, 138, 140], [179, 172,..."


In [10]:
## Subset

map_data_sub = map_data.iloc[0:600,:]
map_data_sub.shape

(600, 2)

## 3. Split data for cross validation

In [11]:
Xs = pd.DataFrame(map_data_sub.iloc[:,1])
ys = pd.DataFrame(map_data_sub.iloc[:,0])
ys['label'] = ys['label'].apply(lambda x: int(x))

In [12]:
# Split data into training data (including training and validation) and test data
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size = 0.2, random_state = 0)

In [13]:
# Split training data into true training data and validation data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)

In [14]:
# confirm the shape and type of our data is right
print('The shape of the first item of X_train is', X_train.iloc[0,0].shape)
print('The length of X_train is', len(X_train))

print('The shape of the first item of X_val is', X_val.iloc[0,0].shape)
print('The length of X_val is', len(X_val))

print('The shape of the first item of X_test is', X_test.iloc[0,0].shape)
print('The length of X_test is', len(X_test))

print('The type and value of the first item of y_train is', type(y_train.iloc[0,0]), y_train.iloc[0,0])
print('The length of y_train is', len(y_train))

print('The shape of the first item of y_val is', type(y_val.iloc[0,0]), y_val.iloc[0,0])
print('The length of y_val is', len(y_val))

print('The type and value of the first item of y_test is', type(y_test.iloc[0,0]), y_test.iloc[0,0])
print('The length of y_test is', len(y_test))

The shape of the first item of X_train is (299, 299, 3)
The length of X_train is 384
The shape of the first item of X_val is (299, 299, 3)
The length of X_val is 96
The shape of the first item of X_test is (299, 299, 3)
The length of X_test is 120
The type and value of the first item of y_train is <class 'numpy.int64'> 4
The length of y_train is 384
The shape of the first item of y_val is <class 'numpy.int64'> 3
The length of y_val is 96
The type and value of the first item of y_test is <class 'numpy.int64'> 4
The length of y_test is 120


In [15]:
# A small check of total data amount before training the model
check_X = (len(X_train) + len(X_val) + len(X_test)) == len(map_data_sub)
check_y = (len(y_train) + len(y_val) + len(y_test)) == len(map_data_sub)

print('X data sets equality is:', check_X)
print('y data sets equality is:', check_y)

X data sets equality is: True
y data sets equality is: True


## 4. One-hot encoding and normalizing our input data

In [16]:
'''# Create combined lists for training and test sets,
# so that we can do normalization and encoding in one function on both data sets.
# Note that 'combine' is only a pointer,
# so when we change something on data sets in combine,
# the original data sets will also change.

combine_X = [X_train, X_val, X_test]
combine_y = [y_train, y_val, y_test]'''

"# Create combined lists for training and test sets,\n# so that we can do normalization and encoding in one function on both data sets.\n# Note that 'combine' is only a pointer,\n# so when we change something on data sets in combine,\n# the original data sets will also change.\n\ncombine_X = [X_train, X_val, X_test]\ncombine_y = [y_train, y_val, y_test]"

In [17]:
'''combine_y = [np.array(y_train['label']), np.array(y_val['label']), np.array(y_test['label'])]'''

"combine_y = [np.array(y_train['label']), np.array(y_val['label']), np.array(y_test['label'])]"

In [18]:
'''# normalize inputs from 0-255 to 0.0-1.0
combine_X = combine_X / 255'''

'# normalize inputs from 0-255 to 0.0-1.0\ncombine_X = combine_X / 255'

In [19]:
'''# check the value of normalized data
print('The shape and value of X data after normalization:', df.iloc[0,0].shape, '\n', df.iloc[0,0][0])'''

"# check the value of normalized data\nprint('The shape and value of X data after normalization:', df.iloc[0,0].shape, '\n', df.iloc[0,0][0])"

In [20]:
## Normalize X

X_train = X_train / 255
X_val = X_val / 255
X_test = X_test / 255

In [21]:
#### from sklearn.preprocessing imposrt LabelEncoder, OneHotEncoder

In [22]:
'''#### one hot encoding our categorical data
#labelencoder_y = LabelEncoder()
#enc = OneHotEncoder()
for df in combine_y:
    df = np_utils.to_categorical(df)
    #df.iloc[:,0] = labelencoder_y.fit_transform(df.iloc[:,0])
    #df = enc.fit(np.array(df).reshape(-1,1))'''

'#### one hot encoding our categorical data\n#labelencoder_y = LabelEncoder()\n#enc = OneHotEncoder()\nfor df in combine_y:\n    df = np_utils.to_categorical(df)\n    #df.iloc[:,0] = labelencoder_y.fit_transform(df.iloc[:,0])\n    #df = enc.fit(np.array(df).reshape(-1,1))'

In [23]:
## One-hot y

y_train = np_utils.to_categorical(y_train)
y_val = np_utils.to_categorical(y_val)
y_test = np_utils.to_categorical(y_test)

In [24]:
'''# Check the type of y after one-hot encoding
type(y_train.iloc[0,0])'''

'# Check the type of y after one-hot encoding\ntype(y_train.iloc[0,0])'

In [25]:
type(y_train)

numpy.ndarray

In [26]:
len(y_train)

384

In [27]:
X_train.shape

(384, 1)

## 5. Model Training

The function below is adapted from: 
http://machinelearningmastery.com/object-recognition-convolutional-neural-networks-keras-deep-learning-library/

In [28]:
# First type of CNN model
input_size=(299,299,3)
num_classes=5

def createCNNModel(num_classes):

    # Create the model
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=input_size, border_mode='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    epochs = 3  # >>> should be 25+
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    return model, epochs

# create our CNN model
model, epochs = createCNNModel(num_classes)
print("CNN Model created.")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 299, 299, 32)      896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 299, 299, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 299, 299, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 710432)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               363741696 
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________

The function below is adapted from: 
http://machinelearningmastery.com/object-recognition-convolutional-neural-networks-keras-deep-learning-library/

In [29]:
'''# Another type of CNN model
input_size=(640,640,3)
num_classes=6

def createCNNModel(num_classes):

    # Create the model
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=input_size, border_mode='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    epochs = 3  # >>> should be 25+
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    return model, epochs

# create our CNN model
model, epochs = createCNNModel(num_classes)
print("CNN Model created.")'''

'# Another type of CNN model\ninput_size=(640,640,3)\nnum_classes=6\n\ndef createCNNModel(num_classes):\n\n    # Create the model\n    model = Sequential()\n    model.add(Convolution2D(32, 3, 3, input_shape=input_size, border_mode=\'same\', activation=\'relu\'))\n    model.add(Dropout(0.2))\n    model.add(MaxPooling2D(pool_size=(2, 2)))\n    model.add(Flatten())\n    model.add(Dense(num_classes, activation=\'softmax\'))\n\n    # Compile model\n    epochs = 3  # >>> should be 25+\n    lrate = 0.01\n    decay = lrate/epochs\n    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)\n    model.compile(loss=\'categorical_crossentropy\', optimizer=sgd, metrics=[\'accuracy\'])\n    print(model.summary())\n    return model, epochs\n\n# create our CNN model\nmodel, epochs = createCNNModel(num_classes)\nprint("CNN Model created.")'

In [30]:
X_train.shape

(384, 1)

In [31]:
X_train_list = X_train['image'].tolist()

In [32]:
X_train_array = np.array(X_train_list)
y_train_array = np.array(y_train)

In [33]:
X_train_array

array([[[[0.57254902, 0.59215686, 0.62745098],
         [0.54509804, 0.56078431, 0.60784314],
         [0.56862745, 0.6       , 0.64313725],
         ...,
         [0.35294118, 0.23529412, 0.21176471],
         [0.34117647, 0.22745098, 0.20784314],
         [0.34901961, 0.23137255, 0.21176471]],

        [[0.59215686, 0.62352941, 0.6627451 ],
         [0.50588235, 0.52941176, 0.60392157],
         [0.49803922, 0.51764706, 0.59607843],
         ...,
         [0.3372549 , 0.22745098, 0.21176471],
         [0.35294118, 0.23137255, 0.21176471],
         [0.35294118, 0.24313725, 0.22352941]],

        [[0.6627451 , 0.69411765, 0.7254902 ],
         [0.50980392, 0.52941176, 0.6       ],
         [0.50980392, 0.52156863, 0.54117647],
         ...,
         [0.34509804, 0.23529412, 0.21960784],
         [0.34117647, 0.22745098, 0.21176471],
         [0.35686275, 0.23137255, 0.20784314]],

        ...,

        [[0.35686275, 0.28235294, 0.2627451 ],
         [0.77647059, 0.7254902 , 0.71764706]

In [34]:
len(X_train_array)

384

In [35]:
y_train_array

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [36]:
#batch size，learning rate can be modified before training

batch_size=60
seed = 7
np.random.seed(seed)
model.fit(X_train_array, y_train_array, batch_size = batch_size, nb_epoch = epochs)
#model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=epochs, batch_size=60)

Epoch 1/3
384/384 [==============================] - 194s 506ms/step - loss: 3.8139 - acc: 0.2734
Epoch 2/3
384/384 [==============================] - 166s 431ms/step - loss: 1.6861 - acc: 0.3724
Epoch 3/3
384/384 [==============================] - 155s 403ms/step - loss: 1.5740 - acc: 0.3490


In [42]:
X_test_array = np.array(X_test['image'].tolist())
y_test_array = np.array(y_test)
X_val_array =np.array(X_val['image'].tolist())
y_val_array = np.array(y_val)

In [43]:
scores = model.evaluate(X_train_array, y_train_array, verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))

scores = model.evaluate(X_val_array, y_val_array, verbose=0)
print("val Accuracy: %.2f%%" % (scores[1]*100))

# Final evaluation of the model
scores = model.evaluate(X_test_array, y_test_array, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

print("done")

Train Accuracy: 39.84%
val Accuracy: 30.21%
Test Accuracy: 40.00%
done


In [ ]:
y_pred=model.predict(X_test)
print("y_pred is ",y_pred)

In [ ]:
y_test

In [ ]:
##store the weights
for layer in model.layers:
    weights = layer.get_weights() 

In [ ]:
weights

In [ ]:
len(weights)

In [ ]:
## store the model
model.save('cnn_trial.h5')

In [ ]:
###read the stored the model
import h5py

def print_keras_wegiths(weight_file_path):
    f = h5py.File(weight_file_path)  # 读取weights h5文件返回File类
    try:
        if len(f.attrs.items()):
            print("{} contains: ".format(weight_file_path))
            print("Root attributes:")
        for key, value in f.attrs.items():
            print("  {}: {}".format(key, value))  # 输出储存在File类中的attrs信息，一般是各层的名称

        for layer, g in f.items():  # 读取各层的名称以及包含层信息的Group类
            print("  {}".format(layer))
            print("    Attributes:")
            for key, value in g.attrs.items(): # 输出储存在Group类中的attrs信息，一般是各层的weights和bias及他们的名称
                print("      {}: {}".format(key, value))  

            print("    Dataset:")
            for name, d in g.items(): # 读取各层储存具体信息的Dataset类
                print("      {}: {}".format(name, d.value.shape)) # 输出储存在Dataset中的层名称和权重，也可以打印dataset的attrs，但是keras中是空的
                print("      {}: {}".format(name. d.value))
    finally:
        f.close()